In [ ]:
import torch

print("GPU disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
!pip install torch torchvision matplotlib scikit-learn timm tqdm


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True,
                                           transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False,
                                          transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=64, shuffle=False)

print("Train size:", len(train_dataset))
print("Test size :", len(test_dataset))


In [ ]:
images, labels = next(iter(train_loader))

fig = plt.figure(figsize=(8,4))
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.imshow(images[i][0], cmap='gray')
    plt.title(f"Label : {labels[i].item()}")
    plt.axis("off")

plt.show()


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128,10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

model = CNN().to(device)
print(model)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5

train_losses = []
train_accs = []

for epoch in range(epochs):
    model.train()
    running_loss = 0
    preds_list = []
    labels_list = []

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.argmax(outputs,1)
        preds_list.extend(preds.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())

    acc = accuracy_score(labels_list, preds_list)
    train_losses.append(running_loss/len(train_loader))
    train_accs.append(acc)

    print(f"Epoch {epoch+1} - Loss: {train_losses[-1]:.4f}, Accuracy: {acc:.4f}")


In [ ]:
model.eval()
preds_list = []
labels_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        preds = torch.argmax(outputs,1)
        preds_list.extend(preds.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())

acc = accuracy_score(labels_list, preds_list)
f1  = f1_score(labels_list, preds_list, average='weighted')

print("CNN TEST ACCURACY:", acc)
print("CNN F1 SCORE     :", f1)


In [ ]:
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn

model_rcnn = fasterrcnn_resnet50_fpn(pretrained=True)
model_rcnn.eval().to(device)

print("Faster R-CNN chargé")


In [ ]:
img, _ = train_dataset[0]
img = img.expand(3, -1, -1)  # convertir en 3 canaux
output = model_rcnn([img.to(device)])

print(output)


In [ ]:
transform_rgb = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset_rgb = torchvision.datasets.MNIST(root='./data', train=True,
                                               transform=transform_rgb, download=True)

test_dataset_rgb = torchvision.datasets.MNIST(root='./data', train=False,
                                              transform=transform_rgb, download=True)

train_loader_rgb = torch.utils.data.DataLoader(train_dataset_rgb, batch_size=64, shuffle=True)
test_loader_rgb  = torch.utils.data.DataLoader(test_dataset_rgb, batch_size=64, shuffle=False)


In [ ]:
from torchvision.models import vgg16

vgg = vgg16(pretrained=True)

for param in vgg.parameters():
    param.requires_grad = False

vgg.classifier[6] = nn.Linear(4096, 10)

vgg = vgg.to(device)
print(vgg.classifier)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.classifier.parameters(), lr=0.0001)

epochs = 3

for epoch in range(epochs):
    vgg.train()
    running_loss = 0

    for images, labels in tqdm(train_loader_rgb):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss : {running_loss/len(train_loader_rgb):.4f}")


In [ ]:
vgg.eval()
preds_list = []
labels_list = []

with torch.no_grad():
    for images, labels in test_loader_rgb:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg(images)

        preds = torch.argmax(outputs,1)
        preds_list.extend(preds.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())

print("VGG16 Accuracy:", accuracy_score(labels_list, preds_list))
print("VGG16 F1 Score:", f1_score(labels_list, preds_list, average='weighted'))


In [ ]:
import timm

vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=10)
vit = vit.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit.parameters(), lr=0.0001)

epochs = 3

for epoch in range(epochs):
    vit.train()
    running_loss = 0

    for images, labels in tqdm(train_loader_rgb):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vit(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss : {running_loss/len(train_loader_rgb):.4f}")


In [ ]:
vit.eval()
preds_list = []
labels_list = []

with torch.no_grad():
    for images, labels in test_loader_rgb:
        images, labels = images.to(device), labels.to(device)
        outputs = vit(images)

        preds = torch.argmax(outputs,1)
        preds_list.extend(preds.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())

print("VIT Accuracy:", accuracy_score(labels_list, preds_list))
print("VIT F1 Score:", f1_score(labels_list, preds_list, average='weighted'))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Résultats réels obtenus
models = ["CNN", "VGG16", "Vision Transformer (ViT)"]
accuracy = [0.9864, 0.9054, 0.9943]
f1_score_vals = [0.9864221958406894, 0.9058810270245607, 0.9942994083319341]

# Création du tableau
df = pd.DataFrame({
    "Modèle": models,
    "Accuracy": accuracy,
    "F1-Score": f1_score_vals
})

print("📊 Tableau comparatif des performances :\n")
display(df)

# Graphique pour Accuracy
plt.figure(figsize=(8,5))
plt.bar(models, accuracy)
plt.title("Comparaison de l'Accuracy des modèles")
plt.xlabel("Modèle")
plt.ylabel("Accuracy")
plt.ylim(0.8, 1.0)
plt.grid(axis='y')
plt.show()

# Graphique pour F1-Score
plt.figure(figsize=(8,5))
plt.bar(models, f1_score_vals)
plt.title("Comparaison du F1-Score des modèles")
plt.xlabel("Modèle")
plt.ylabel("F1-Score")
plt.ylim(0.8, 1.0)
plt.grid(axis='y')
plt.show()
